In [1]:
import cv2
import numpy as np
import math
import colorsys
import csv

In [2]:
#ランダムで座標と値を発生
#本番はリストで受け取るので、この関数は不要
def getRandom2DPoints(width, height, n):
    with open('sample.csv','r') as f:
        reader = csv.reader(f)
        pts = np.zeros((50,3))
        i = 0
        for line in reader:
            if(i>0):
                pts[i-1][0]=line[0]
                pts[i-1][1]=line[1]
                pts[i-1][2]=line[2]
            i=i+1
        return pts

In [3]:
def get_color2(a1,a2,a3):
    #平均値を求める
    ave = (a1+a2+a3)/3
    if (ave>=0):
        r = 255
        g = 255 - ave*5
        b = 255 - ave*5
    if (ave<0):
        r = 255 + ave*5
        g = 255 + ave*5
        b = 255
    return(b,g,r)

In [4]:
def DelaunayDiagram(imgd, subdiv,array):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    
    #rgb値を格納する配列を作成
    color_list = np.zeros((len(inner_pts),3))
    
    #3値を渡して色を設定
    for i in range(len(inner_pts)):
        color_list[i]=get_color2(array[(int)(inner_pts[i][0][0])][(int)(inner_pts[i][0][1])],array[(int)(inner_pts[i][1][0])][(int)(inner_pts[i][1][1])],array[(int)(inner_pts[i][2][0])][(int)(inner_pts[i][2][1])])
    #図形の描画
    for i in range(len(inner_pts)):
        #三角形の描画
        cv2.fillPoly(imgd, [inner_pts[i].astype(int)], (color_list[i][0],color_list[i][1],color_list[i][2]), 1)
        #三角形の辺の描画
        cv2.line(imgd,(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(0,0,0))

In [10]:
def main():
    width = 320
    height = 240
    
    #点の生成
    pts = getRandom2DPoints(width,height,50)
    print(pts)
    #白紙を作る
    img = np.full((height, width, 3), 255, np.uint8)
    
    rect = (0, 0, width, height)
    subdiv = cv2.Subdiv2D(rect)

    #(x1,y1,a1)の時、array[x1][y1]=a1とする
    array = np.zeros((width,height))
    #print(array)
    
    for p in pts:
        subdiv.insert((int(p[0]), int(p[1])))
        array[int(p[0])][int(p[1])] = p[2]
        
    DelaunayDiagram(img, subdiv,array)

    for p in pts:
        #color = p[2]
        cv2.circle(img, (p[0],p[1]), 4, (0, 0, 0), thickness=-1, lineType=cv2.LINE_AA)
   
    cv2.imwrite('test2.png', img)

In [11]:
if __name__ == '__main__':
    main()

[[179. 184.  39.]
 [166. 223.  25.]
 [185.  72.  29.]
 [ 70. 165.  35.]
 [ 47. 112.  -1.]
 [224.  53.   8.]
 [122.  78.  -7.]
 [231.  29.  29.]
 [ 49. 122.  35.]
 [ 14. 189.   2.]
 [239.  71.  28.]
 [ 87. 167.  40.]
 [ 72. 144.  13.]
 [115. 219.  25.]
 [160.  89.  41.]
 [182. 151.  40.]
 [180.   7.  28.]
 [ 10.  97.  -3.]
 [295.   5.  42.]
 [188. 209.  33.]
 [  6.  54.  16.]
 [ 88. 174.  16.]
 [241. 224.  14.]
 [ 47. 104.  25.]
 [292.  82.  42.]
 [142.  55.   9.]
 [218. 114.  -4.]
 [ 60. 138.  18.]
 [104.  32.  14.]
 [ 88.  44.  43.]
 [ 91.  68.  43.]
 [226.  75.  12.]
 [105.  34.  34.]
 [251. 102.  37.]
 [ 45.  54.  21.]
 [135. 212.   3.]
 [250.  28.  37.]
 [116.  54.   1.]
 [244.  90.  23.]
 [205.  39.  14.]
 [ 52. 108.   0.]
 [196. 186.   9.]
 [ 50.  58.  -6.]
 [188. 123.  39.]
 [259. 129.  -4.]
 [193. 221. -10.]
 [  1.  44.  16.]
 [264.  95.   8.]
 [ 53. 159.  22.]
 [154.  45.  11.]]


error: OpenCV(4.5.4-dev) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
